In [1]:
import pandas as pd

In [2]:
# setup plotting
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv",
                   parse_dates=['date'],
                   index_col=['state', 'county', 'date'])

data = data.sort_index()  # avoid PerformanceWarning:indexing past lexsort depth may impact performance.
data

HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
# New Cases: Difference between current day and prior day total reported cases
data['new cases'] = data.groupby(['state', 'county'])['cases'].diff()

In [ ]:
# Recovered Cases: Assuming 14 day recovery period, total cases reported 14 days prior less any deaths.
recovery_period = 14  # days
shifted_cases = data.groupby(['state', 'county'])['cases'].shift(recovery_period)
#shifted_cases
data['recovered cases'] = data.apply(lambda row: shifted_cases[row.name] - row['deaths'], axis=1)

In [ ]:
# Active Cases: Difference between current day total reported cases less reported deaths and recovered cases.
data['active cases'] = data.apply(lambda row: row['cases'] - row['deaths'] - row['recovered cases'], axis=1)

In [ ]:
state = 'Texas'
county = 'Travis'

In [ ]:
county_data = data.loc[(state, county)][['cases', 'new cases', 'deaths', 'recovered cases', 'active cases']]
county_data

In [ ]:
county_data.iplot(title=f"{county} County, {state}")